In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Getting stocks and ETFs based on this article "https://www.nasdaq.com/articles/generative-ai-is-as-transformative-as-the-telephone-bank-of-america-says:-10-stock-etf"
tickers = ['GOOGL','MSFT','AMZN','META','CRM','NVDA','SMCI','INTC','AMD','AAPL']

## Company Information

### Create DataFrame to store Sector, Price to Earnings Ratio, Company Beta

In [7]:
# Create an empty DataFrame to store the data
stock_data = pd.DataFrame(columns=['Ticker','CompanyName' ,'Sector', 'TrailingPE', 'Beta'])

for stock_ticker in tickers:
    stock_info = yf.Ticker(stock_ticker).info
    
    # Extract information
    sector = stock_info.get('sector', 'N/A')
    trailingPE = stock_info.get('trailingPE', 'N/A')
    beta = stock_info.get('beta', 'N/A')
    
    # Append data to the DataFrame
    stock_data = pd.concat([stock_data, pd.DataFrame({
        'Ticker': [stock_ticker],
        'CompanyName': [longName],
        'Sector': [sector],
        'TrailingPE': [trailingPE],
        'Beta': [beta]
    })], ignore_index=True)

    # Display information (optional)
    print(f"{stock_ticker} - Sector: {sector}, TrailingPE: {trailingPE}, Beta: {beta}")

# Save DataFrame to CSV
stock_data.to_csv('stock_data.csv', index=False)

C:\Users\Andrew\AppData\Local\Temp\ipykernel_2376\2101541996.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stock_data = pd.concat([stock_data, pd.DataFrame({


GOOGL - Sector: Communication Services, TrailingPE: 26.2318, Beta: 1.035
MSFT - Sector: Technology, TrailingPE: 35.97963, Beta: 0.897
AMZN - Sector: Consumer Cyclical, TrailingPE: 76.90053, Beta: 1.17
META - Sector: Communication Services, TrailingPE: 28.825243, Beta: 1.197
CRM - Sector: Technology, TrailingPE: 94.26136, Beta: 1.24
NVDA - Sector: Technology, TrailingPE: 61.310528, Beta: 1.691
SMCI - Sector: Technology, TrailingPE: 23.630074, Beta: 1.149
INTC - Sector: Technology, TrailingPE: N/A, Beta: 0.95
AMD - Sector: Technology, TrailingPE: 1069.75, Beta: 1.674
AAPL - Sector: Technology, TrailingPE: 31.640066, Beta: 1.308


## Closing Price

### Create DataFrame to store closing prices

In [8]:
for ticker in tickers:
    data = yf.download(ticker, group_by="Ticker")
    data['ticker'] = ticker  # add this column because the dataframe doesn't contain a column with the ticker
    data.to_csv(f'ticker_{ticker}.csv')  # ticker_AAPL.csv for example


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
